# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

603

In [6]:
print(cities)

['illoqqortoormiut', 'albany', 'butaritari', 'severo-kurilsk', 'barrow', 'airai', 'pisco', 'seoul', 'mataura', 'ilhabela', 'rikitea', 'taolanaro', 'chuy', 'cape town', 'east london', 'atar', 'grand river south east', 'new norfolk', 'richards bay', 'isangel', 'nouadhibou', 'jalu', 'mys shmidta', 'tuktoyaktuk', 'atuona', 'amasya', 'qaanaaq', 'belushya guba', 'port hedland', 'cheyur', 'cherskiy', 'egvekinot', 'abu samrah', 'thompson', 'penzance', 'nanortalik', 'khatanga', 'almaznyy', 'jieshi', 'moron', 'busselton', 'bambous virieux', 'la libertad', 'lahuachaca', 'buqayq', 'kodiak', 'safwah', 'mana', 'smithers', 'tiksi', 'anchorage', 'kapaa', 'hermanus', 'camalu', 'bossembele', 'ushuaia', 'saint george', 'faanui', 'bredasdorp', 'punta arenas', 'pacific grove', 'havre-saint-pierre', 'hithadhoo', 'hobyo', 'marrakesh', 'mar del plata', 'pundaguitan', 'talnakh', 'karkaralinsk', 'mayumba', 'castro', 'mahibadhoo', 'jamestown', 'labuan', 'bethel', 'bartica', 'te anau', 'alice springs', 'vanimo', 

In [30]:
cities_20 = cities[0:20]
print(cities_20)
print(len(cities_20))

['lolua', 'upernavik', 'antofagasta', 'jalu', 'tsihombe', 'dalvik', 'hilo', 'albany', 'rio grande', 'praia da vitoria', 'peniche', 'taolanaro', 'guane', 'chicama', 'nantucket', 'shelburne', 'san angelo', 'kodiak', 'busselton', 'puerto ayora']
20


In [49]:
df = pd.DataFrame(cities_20)
df=df.rename(columns={0:'City'})
df

,City
0,lolua
1,upernavik
2,antofagasta
3,jalu
4,tsihombe
5,dalvik
6,hilo
7,albany
8,rio grande
9,praia da vitoria


In [51]:
df['lat'] = ""
df['long'] = ""
df['temp'] = ""
df['humidity'] = ""
df['cloudiness'] = ""
df['windspeed'] = ""
df

,City,lat,long,temp,humidity,cloudiness,windspeed
0,lolua,,,,,,
1,upernavik,,,,,,
2,antofagasta,,,,,,
3,jalu,,,,,,
4,tsihombe,,,,,,
5,dalvik,,,,,,
6,hilo,,,,,,
7,albany,,,,,,
8,rio grande,,,,,,
9,praia da vitoria,,,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [53]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units="metric"
api_key = weather_api_key

query_url = f"{url}appid={api_key}&units={units}&q="

#city=cities
#city=cities_20


In [56]:
for i, row in df.iterrows():
    city=row[0]
    
    response = requests.get(query_url + city).json()
    try:
        lat = response['coord']['lat']
        long = response['coord']['lon']
        temp = response['main']['temp']
        humidity = response['main']['humidity']
        cloudiness = response['clouds']['all']
        windspeed = response['wind']['speed']
           
        df.loc[i, "lat"] = lat
        df.loc[i, "long"] = long
        df.loc[i, "temp"] = temp
        df.loc[i, "humidity"] = humidity
        df.loc[i, "cloudiness"] = cloudiness
        df.loc[i, "windspeed"] = windspeed
    except:
        print(city)
        print(response)

df.head(20)

lolua
{'cod': '404', 'message': 'city not found'}
tsihombe
{'cod': '404', 'message': 'city not found'}
taolanaro
{'cod': '404', 'message': 'city not found'}


,City,lat,long,temp,humidity,cloudiness,windspeed
0,lolua,,,,,,
1,upernavik,72.7868,-56.1549,-1.88,91,100,4.14
2,antofagasta,-23.65,-70.4,16,77,90,1.03
3,jalu,29.0331,21.5482,36.27,9,0,1.02
4,tsihombe,,,,,,
5,dalvik,65.9702,-18.5286,5,52,0,2.06
6,hilo,19.7297,-155.09,19.17,73,90,0.49
7,albany,42.6001,-73.9662,8.45,61,84,5.84
8,rio grande,-32.035,-52.0986,9.42,81,0,3.09
9,praia da vitoria,38.7333,-27.0667,19.19,55,0,2.42


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [5]:
#  Get the indices of cities that have humidity over 100%.


In [6]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression